## Privacy-preserving Fake News Detection
**Universidade de Brasília**<br>
School of Technology<br>
Graduate Program in Electrical Engineering (PPGEE)

### Author: Stefano M P C Souza (stefanomozart@ieee.org)<br> Author: Daniel G Silva<br>Author: Anderson C A Nascimento

# Clear-text Neural Network Benchmark

Our general goal in this research work is to demonstrate the use of secure Multi-party Computation (MPC) protocols in order to provide privacy-preserving fake news detection techniques. We are going to use neural networks inference models to classify texts. The MPC protocols can be used both during the training and inference phases. 

In this notebook we train and test these neural networks in the clear-text setting --- that is, without any atempt cipher or keep the confidentiality of the datasets or models. We are going to use these results as a benchmark, in order to study the impact of running the same algorithms upon MPC protocols on the performance of the predictive models.

**Notice** that you need to run the [Embeddings](./embeddings.ipynb) notebook beforehand, in order to encode the datasets with the chosen BERT embeddings.

**Concerning reproducibility:**

We make our best effort to present reproducible experiments, however, it cannot be guaranteed across PyTorch and CrypTen non-deterministic algorithms. Also, the cuDNN library, used by CUDA convolution operations, can be a source of nondeterminism across multiple executions.

In [1]:
# Utilities
import os, sys, time, random
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [2]:
# PyTorch
import torch

torch.cuda.empty_cache()
torch.manual_seed(42)

In [3]:
# Path to our models
sys.path.insert(0, sys.path.insert(0, os.path.abspath('../')))

# Our Convolutional Neural Networks
from models.cnn import CNN, CNN2, CNN3, CNN4, CNN5

# Our Deep Feed-Forward Neural Network
from models.fnn import FNN, FNN2, FNN3, FNN4, FNN5

# The CNN from [1]
from models.adams import BenchmarkCNN

models = [FNN, FNN2, FNN3, FNN4, FNN5, CNN, CNN2, CNN3, CNN4, CNN5, BenchmarkCNN]

In [4]:
#- Experiment setup

NUM_EPOCHS = 10
LEARNING_RATE = 0.01
BATCH_SIZE = 50
OUTPUT_PATH = './output'
DATASET_HOME="/home/ppml/datasets"
DATASETS = ["liar", "sbnc", "fake.br", "factck.br"]

# Check the Embeddings (embeddings.ipynb) notebook to see how the datasets were encoded to embeddings)
EMBEDDINGS = ["stsb-distilbert-base", "paraphrase-multilingual-mpnet-base-v2"]

## Training and testing

The following functions are used to load the datasets and train the neural networks.

In [5]:
# Model training
def train(model, model_name, optimizer, dataset, num_epochs, output_dir="", save_model=True):
    # Create output dir, if it does not exist
    if save_model:
        os.makedirs(output_dir + 'models', exist_ok=True)

    best_loss = 1
    
    # Training
    batch_index = [i for i in range(len(dataset['train']))]
    t = time.process_time()
    for epoch in range(num_epochs):
        random.shuffle(batch_index)
    
        model.train()
        losses = []
        criterion = torch.nn.CrossEntropyLoss()
        
        # We are going to train the model with batches of BATCH_SIZE elements
        for i in batch_index:
            optimizer.zero_grad()
            probs = model(dataset['train'][i])
            loss = criterion(probs, dataset['train_label'][i])
            losses.append(loss.item() / len(dataset['train'][i]))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
        
        epoch_loss = np.mean(losses)
        
        info = test(model, dataset['valid'], dataset['valid_label'])

        if save_model and info['loss'] < best_loss:
            torch.save(model, f"{output_dir}{model_name}.best.model")
            torch.save(info, f"{output_dir}{model_name}.best.info")
            best_loss = info['loss']

    train_runtime = time.process_time() - t

    if save_model:
        torch.save(model, f"{output_dir}{model_name}.last.model")

    # Model test
    t = time.process_time()
    info = test(model, dataset['test'], dataset['test_label'])    
    info['train_runtime'] = train_runtime 
    info['test_runtime'] = time.process_time() - t
    print(f" -- Training results: accuracy={info['accuracy']}, runtime={train_runtime}")

    return info  

In [6]:
# Model evaluation: Cross-entropy losss, accuracy, F1-score and ROC AUC score
def test(model, X, y):
    model.eval()
    losses = []
    y_predicted = []
    y_true = []
    
    for bX, by in zip(X, y):
        #embeddings, labels = tuple(i.to(device) for i in batch)

        with torch.no_grad():
            probs = model(bX)

        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(probs, by)
        
        losses.append(loss.item()/len(bX))
        
        predicted = probs.max(1).indices
        
        y_predicted.extend(predicted.tolist())
        y_true.extend(by.tolist())

    return {
        'loss': np.mean(losses), 
        'accuracy': accuracy_score(y_true, y_predicted), 
        'f1_score': f1_score(y_true, y_predicted),
        'roc_auc': roc_auc_score(y_true, y_predicted)
    }

In [7]:
# Load a numpy array, convert to torch tensor and split in batches
def load_torch_split(path, dtype=None, batch_size=BATCH_SIZE):
    arr = np.load(path, allow_pickle=True)
    ten = torch.tensor(arr, dtype=dtype)
    return torch.split(ten, batch_size)

## Experiments 

In [8]:
# Load dataset labels
datasets = {}
for d in DATASETS:
    dtpath = f'{DATASET_HOME}/{d}'
    
    datasets[d] = {
        'name': d,
        'train_label': load_torch_split(f"{dtpath}/train.labels.npy", dtype=torch.long),
        'valid_label': load_torch_split(f"{dtpath}/valid.labels.npy", dtype=torch.long),
        'test_label': load_torch_split(f"{dtpath}/test.labels.npy", dtype=torch.long)
    }    


In [9]:
from torch.optim import AdamW

# load previous results, to add new experimental results
try:
    results = joblib.load('cleartext_benchmark.pyd')
except FileNotFoundError:
    results = pd.DataFrame()
    joblib.dump(results, 'cleartext_benchmark.pyd')

for d in datasets.keys():
    dtpath = f'{DATASET_HOME}/{d}'
    
    
    for e in EMBEDDINGS:
        datasets[d]['train'] = load_torch_split(f"{dtpath}/train.{e}.npy")
        datasets[d]['valid'] = load_torch_split(f"{dtpath}/valid.{e}.npy")
        datasets[d]['test'] = load_torch_split(f"{dtpath}/test.{e}.npy")

        output_path = f'{OUTPUT_PATH}/{d}/{e}/'
        
        for m in models:
            print("\n---------------------------------------------------------------------------------------")
            print(d, e, m.__name__)

            mdl = m()
            mdl_info = train(
                mdl,                                       # Model instance
                m.__name__,                                # Model name (to save)
                AdamW(mdl.parameters(), lr=LEARNING_RATE), # Optimizer
                datasets[d],                               # Dataset with train, valid and test sets
                NUM_EPOCHS,                                # number of epochs for training
                output_path                                # path to save trained models
            )

            mdl_info['dataset'] = d
            mdl_info['embedding'] = e
            mdl_info['model'] = m.__name__
            
            results = results.append(mdl_info, ignore_index=True)


---------------------------------------------------------------------------------------
liar stsb-distilbert-base FNN
 -- Training results: accuracy=0.5576397030089879, runtime=47.723126609999994

---------------------------------------------------------------------------------------
liar stsb-distilbert-base FNN2
 -- Training results: accuracy=0.5576397030089879, runtime=29.848463197

---------------------------------------------------------------------------------------
liar stsb-distilbert-base FNN3
 -- Training results: accuracy=0.5576397030089879, runtime=24.561286620999994

---------------------------------------------------------------------------------------
liar stsb-distilbert-base FNN4
 -- Training results: accuracy=0.5576397030089879, runtime=21.834679205

---------------------------------------------------------------------------------------
liar stsb-distilbert-base FNN5
 -- Training results: accuracy=0.5849941383352872, runtime=8.771627180999985

-----------------------

 -- Training results: accuracy=0.6757425742574258, runtime=13.970093046000102

---------------------------------------------------------------------------------------
sbnc paraphrase-multilingual-mpnet-base-v2 CNN4
 -- Training results: accuracy=0.6039603960396039, runtime=15.638607274000151

---------------------------------------------------------------------------------------
sbnc paraphrase-multilingual-mpnet-base-v2 CNN5
 -- Training results: accuracy=0.6039603960396039, runtime=15.419316557000002

---------------------------------------------------------------------------------------
sbnc paraphrase-multilingual-mpnet-base-v2 BenchmarkCNN
 -- Training results: accuracy=0.6039603960396039, runtime=12.223547996999969

---------------------------------------------------------------------------------------
fake.br stsb-distilbert-base FNN
 -- Training results: accuracy=0.5, runtime=27.754292634999956

-----------------------------------------------------------------------------------

 -- Training results: accuracy=0.7908745247148289, runtime=1.088542069000141

---------------------------------------------------------------------------------------
factck.br paraphrase-multilingual-mpnet-base-v2 CNN
 -- Training results: accuracy=0.779467680608365, runtime=5.414315801000157

---------------------------------------------------------------------------------------
factck.br paraphrase-multilingual-mpnet-base-v2 CNN2
 -- Training results: accuracy=0.779467680608365, runtime=9.906651195999984

---------------------------------------------------------------------------------------
factck.br paraphrase-multilingual-mpnet-base-v2 CNN3
 -- Training results: accuracy=0.779467680608365, runtime=8.780549404999874

---------------------------------------------------------------------------------------
factck.br paraphrase-multilingual-mpnet-base-v2 CNN4
 -- Training results: accuracy=0.7680608365019012, runtime=10.547003724000206

-------------------------------------------------

### Best accuracy on the test set, for each dataset

In [10]:
results.sort_values(by=['accuracy'], ascending=False).groupby(by='dataset').first()

,accuracy,embedding,f1_score,loss,model,roc_auc,test_runtime,train_runtime
dataset,,,,,,,,
factck.br,0.821293,paraphrase-multilingual-mpnet-base-v2,0.891455,0.018820,CNN4,0.670201,0.093772,9.921069
fake.br,0.826389,paraphrase-multilingual-mpnet-base-v2,0.828297,0.008419,CNN,0.826389,0.209264,27.105803
liar,0.613521,paraphrase-multilingual-mpnet-base-v2,0.586711,0.014161,CNN,0.614205,0.431463,52.808952
sbnc,0.737624,paraphrase-multilingual-mpnet-base-v2,0.804428,0.021291,CNN2,0.696721,0.128258,14.871974


### Best ROC AUC

In [11]:
results.sort_values(by=['roc_auc'], ascending=False).groupby(by='dataset').first()

,accuracy,embedding,f1_score,loss,model,roc_auc,test_runtime,train_runtime
dataset,,,,,,,,
factck.br,0.775665,paraphrase-multilingual-mpnet-base-v2,0.852868,0.033022,CNN,0.704522,0.061667,5.629209
fake.br,0.826389,paraphrase-multilingual-mpnet-base-v2,0.828297,0.008419,CNN,0.826389,0.209264,27.105803
liar,0.613521,paraphrase-multilingual-mpnet-base-v2,0.586711,0.014161,CNN,0.614205,0.431463,52.808952
sbnc,0.727723,paraphrase-multilingual-mpnet-base-v2,0.773663,0.099958,FNN5,0.716496,0.017222,1.630381


### Best F1-score

In [12]:
results.sort_values(by=['f1_score'], ascending=False).groupby(by='dataset').first()

,accuracy,embedding,f1_score,loss,model,roc_auc,test_runtime,train_runtime
dataset,,,,,,,,
factck.br,0.821293,paraphrase-multilingual-mpnet-base-v2,0.891455,0.018820,CNN4,0.670201,0.093772,9.921069
fake.br,0.826389,paraphrase-multilingual-mpnet-base-v2,0.828297,0.008419,CNN,0.826389,0.209264,27.105803
liar,0.575615,paraphrase-multilingual-mpnet-base-v2,0.611310,0.014434,CNN,0.594097,0.411986,49.048951
sbnc,0.737624,paraphrase-multilingual-mpnet-base-v2,0.804428,0.021291,CNN2,0.696721,0.128258,14.871974


### Average metrics

In [13]:
results['avg'] = (results.accuracy+results.roc_auc+results.f1_score)/3
results.sort_values(by=['avg'], ascending=False).groupby(by='dataset').first()

,accuracy,embedding,f1_score,loss,model,roc_auc,test_runtime,train_runtime,avg
dataset,,,,,,,,,
factck.br,0.821293,paraphrase-multilingual-mpnet-base-v2,0.891455,0.018820,CNN4,0.670201,0.093772,9.921069,0.794316
fake.br,0.826389,paraphrase-multilingual-mpnet-base-v2,0.828297,0.008419,CNN,0.826389,0.209264,27.105803,0.827025
liar,0.613521,paraphrase-multilingual-mpnet-base-v2,0.586711,0.014161,CNN,0.614205,0.431463,52.808952,0.604812
sbnc,0.737624,paraphrase-multilingual-mpnet-base-v2,0.804428,0.021291,CNN2,0.696721,0.128258,14.871974,0.746258


### Average metrics, by dataset and model

In [14]:
results.groupby(by=['dataset', 'model']).mean().sort_values(by=['dataset', 'avg'], ascending=False)

accuracy  f1_score      loss   roc_auc  test_runtime  \
dataset   model                                                                
sbnc      FNN5          0.692915  0.748660  0.066990  0.675173      0.019405   
          CNN           0.665687  0.710086  0.036213  0.657137      0.079116   
          CNN3          0.650062  0.718303  0.031806  0.625845      0.142188   
          CNN2          0.653929  0.718829  0.038481  0.615263      0.130994   
          CNN5          0.638769  0.721105  0.032030  0.601098      0.162941   
          FNN4          0.624536  0.712879  0.030326  0.569278      0.029512   
          CNN4          0.618193  0.737488  0.038223  0.541099      0.132619   
          FNN3          0.615254  0.730508  0.031378  0.542364      0.028126   
          FNN           0.610613  0.753654  0.031611  0.512164      0.040277   
          BenchmarkCNN  0.604785  0.753475  0.031862  0.501042      0.094214   
          FNN2          0.603960  0.753086  0.031774  0.500000      0.033897   
liar      FNN5          0.580842  0.490292  0.024747  0.568360      0.078489   
          CNN           0.579719  0.291433  0.014723  0.552366      0.394101   
          CNN3          0.561059  0.203378  0.014970  0.523040      0.822504   
          CNN2          0.570291  0.175010  0.014900  0.530296      0.745290   
          CNN5          0.559838  0.103942  0.014899  0.512061      0.959909   
          FNN4          0.557664  0.000221  0.015027  0.500033      0.119283   
          FNN3          0.557640  0.000110  0.014993  0.500006      0.114629   
          BenchmarkCNN  0.557640  0.000000  0.014989  0.500000      0.535964   
          CNN4          0.557640  0.000000  0.015006  0.500000      0.757785   
          FNN           0.557640  0.000000  0.015004  0.500000      0.188871   
          FNN2          0.557640  0.000000  0.014996  0.500000      0.142424   
fake.br   FNN5          0.794531  0.789484  0.011423  0.794531      0.052664   
          CNN           0.765972  0.765500  0.009787  0.765972      0.238240   
          CNN2          0.697005  0.682179  0.011305  0.697005      0.423810   
          FNN4          0.651649  0.609645  0.011696  0.651649      0.074928   
          CNN5          0.627257  0.524712  0.012500  0.627257      0.505793   
          CNN3          0.603950  0.560710  0.012781  0.603950      0.431403   
          FNN3          0.645182  0.433268  0.012095  0.645182      0.073715   
          CNN4          0.611719  0.341141  0.012787  0.611719      0.415515   
          FNN2          0.500000  0.416667  0.013992  0.500000      0.084900   
          BenchmarkCNN  0.500000  0.333333  0.013988  0.500000      0.298377   
          FNN           0.500000  0.333333  0.013997  0.500000      0.107129   
factck.br FNN5          0.762357  0.850416  0.029107  0.627028      0.017001   
          CNN           0.765888  0.853715  0.023617  0.617159      0.056402   
          CNN2          0.770675  0.858799  0.018990  0.602993      0.090091   
          CNN3          0.769962  0.859103  0.016517  0.600952      0.098912   
          CNN5          0.763546  0.855289  0.017509  0.591304      0.104485   
          CNN4          0.763783  0.853605  0.032436  0.577973      0.090803   
          FNN4          0.767348  0.859698  0.018346  0.564387      0.024248   
          BenchmarkCNN  0.783270  0.878465  0.014286  0.500000      0.067015   
          FNN           0.783270  0.878465  0.014488  0.500000      0.032353   
          FNN2          0.783270  0.878465  0.014415  0.500000      0.026404   
          FNN3          0.749762  0.837730  0.022224  0.563868      0.021984   

                        train_runtime       avg  
dataset   model                                  
sbnc      FNN5               1.723053  0.705582  
          CNN                8.565199  0.677637  
          CNN3              13.383544  0.664737  
          CNN2              14.816163  0.662674  
          CNN5              15.710287  0.65

### Average metrics, by model

In [15]:
results.groupby(by=['model']).mean().sort_values(by=['avg'], ascending=False)

,accuracy,f1_score,loss,roc_auc,test_runtime,train_runtime,avg
model,,,,,,,
FNN5,0.707661,0.719713,0.033067,0.666273,0.041890,4.602997,0.697882
CNN,0.692008,0.648779,0.021003,0.649159,0.196338,23.440761,0.663315
CNN2,0.672975,0.608704,0.020919,0.611389,0.347546,40.825454,0.631023
CNN3,0.646258,0.585373,0.019018,0.588447,0.373752,35.670399,0.606693
CNN5,0.647352,0.551262,0.019235,0.582930,0.433282,42.396833,0.593848
FNN4,0.650299,0.545611,0.018849,0.571337,0.061992,11.198415,0.589082
FNN3,0.641960,0.500404,0.020173,0.562855,0.059613,11.706963,0.568406
CNN4,0.637834,0.483058,0.024613,0.557698,0.349181,43.531000,0.559530
FNN2,0.611218,0.512054,0.018794,0.500000,0.071906,16.523024,0.541091


## Models selected for PPML experiments

As the results above show that the CNN and the FNN5 models appear as the top average metrics for most datasets, we are going to use these models in the next step of our resarch. 

## Comparisson with bechmark CNN from literature

Below are the results for the CNN found in literature [1]. One can note that it does not appear in the best position for any of the metrics, as displayed above.

In [16]:
results[results.model=='BenchmarkCNN'].groupby(by=['dataset', 'embedding']).mean().sort_values(by=['accuracy'], ascending=False)

accuracy  f1_score      loss  \
dataset   embedding                                                             
factck.br paraphrase-multilingual-mpnet-base-v2  0.783270  0.878465  0.014426   
          stsb-distilbert-base                   0.783270  0.878465  0.014147   
sbnc      stsb-distilbert-base                   0.605611  0.753864  0.031897   
          paraphrase-multilingual-mpnet-base-v2  0.603960  0.753086  0.031827   
liar      paraphrase-multilingual-mpnet-base-v2  0.557640  0.000000  0.014993   
          stsb-distilbert-base                   0.557640  0.000000  0.014985   
fake.br   paraphrase-multilingual-mpnet-base-v2  0.500000  0.222222  0.013986   
          stsb-distilbert-base                   0.500000  0.444444  0.013990   

                                                  roc_auc  test_runtime  \
dataset   embedding                                                       
factck.br paraphrase-multilingual-mpnet-base-v2  0.500000      0.064148   
          stsb-distilbert-base                   0.500000      0.069881   
sbnc      stsb-distilbert-base                   0.502083      0.092576   
          paraphrase-multilingual-mpnet-base-v2  0.500000      0.095853   
liar      paraphrase-multilingual-mpnet-base-v2  0.500000      0.551043   
          stsb-distilbert-base                   0.500000      0.520885   
fake.br   paraphrase-multilingual-mpnet-base-v2  0.500000      0.294154   
          stsb-distilbert-base                   0.500000      0.302599   

                                                 train_runtime       avg  
dataset   embedding                                                       
factck.br paraphrase-multilingual-mpnet-base-v2       7.672710  0.720578  
          stsb-distilbert-base                        7.794583  0.720578  
sbnc      stsb-distilbert-base                       12.166000  0.620519  
          paraphrase-multilingual-mpnet-base-v2      12.188628  0.619016  
liar      paraphrase-multilingual-mpnet-base-v2      71.854150  0.352547  
          stsb-distilbert-base                       71.356444  0.352547  
fake.br   paraphrase-multilingual-mpnet-base-v2      40.256484  0.407407  
          stsb-distilbert-base                       40.171994  0.481481

In [17]:
joblib.dump(results, 'cleartext_benchmark.pyd')

['cleartext_benchmark.pyd']

## References

[1] S. Adams, D. Melanson, M. De Cock, Private text classification with convolutional neural networks, in: Proceedings of the Third Workshop on Privacy in Natural Language Processing, Association for Computational Linguistics, Online, 2021, pp. 53–58. doi:10.18653/v1/2021.privatenlp-1.7.
